# <span style="color:#F72585">Pytorch-lightning</span>

<figure>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro_Fundamentos_Programacion/main/Pytorch/Imagenes/Pytorch_ligthning_logo.png" width="200" align="left" />   
</figure>

## <span style="color:blue">Introducción</span> 

Tomado de [Wikipedia](https://en.wikipedia.org/wiki/PyTorch_Lightning)

PyTorch Lightning es una biblioteca Python de código abierto que proporciona una interfaz de alto nivel para PyTorch , un marco de aprendizaje profundo popular. Es un marco liviano y de alto rendimiento que organiza el código PyTorch para desvincular la investigación de la ingeniería, lo que hace que los experimentos de aprendizaje profundo sean más fáciles de leer y reproducir. Está diseñada para crear modelos de aprendizaje profundo escalables que pueden ejecutarse fácilmente en hardware distribuido y mantener los modelos independientes del hardware.

En 2019, Lightning fue adoptado por NeurIPS Reproducibility Challenge como estándar para enviar el código PyTorch a la conferencia.

## <span style="color:blue">Entrenamiento de un modelo con Pythorch-lightning</span> 

La siguiente imagen representa los diferentes objetos requeridos para entrenar un modelo supervisado de redes neuronales. En general cada uno de estos objetos está presente en cualquier marco de trabajo (framework) para el entrenamiento de una red neuronal. Estos componentes son:

<figure>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro_Fundamentos_Programacion/main/Pytorch/Imagenes/trainer.png" width="800" heigh="600" valign="left" />   
</figure>

Fuente: Alvaro Montenegro

### <span style="color:#4CC9F0">Datos: entrenamiento, validación, prueba</span>

Para entrenar un modelo supérvisado necesitamos datos. Por lo gneral los datos se separan en dos o tres grupos que llamamos:

* `datos de entrenamiento`. Usualmente 70% u 80% del toptal de datos, elegido al azar y que son los datos que verá el optimizador en el proceso de entrenamiento; 
* `datos de validación`. Usualmente 10% 0 20%. Son datos usadps durante el prceso de entranamiento para validacion en linea del modelo mientras es entrenado. Usualmente se usan con la función de pérdida y con la métricas definidas en cada caso;
* `datos de prueba`. Son datos externos al entrenamiento. Usualmente 10% del total de dato y se usan un vez ha terminado del entrenamiento para evaluar si el modelo generaliza adecuadamente, es decir si tiene el poder prodictivo que se espera. 

### <span style="color:#4CC9F0">Modelo: la red neuronal</span>

Es la red neuronal que se ha diseñado para resolver el problema que se tiene entre manos. Usualmente cosnstruimos una clase en la cual de defines los componentes que tendrá la red y que serás conjuntos de capas separadas de manera lógica. En Pytorch el modelo se implementa con el método `foreward`. En este método se determina exáctamente como funciona la red neuronal. Es el método de cálculo de la red neuronal. 

### <span style="color:#4CC9F0">Entrenador: pérdida optimizador, métricas</span>

Es el objeto que se encarga de hacer el entrenamiento de la red. El entrenador (trainer) usualmente tiene dos componentes escenciales:

* `paso de optimización` en el cual se define lo que se debe hace en cada paso del proceso de optimización;
* `ciclo de entrenamiento` que define como ocurrirá tos el entrenamiento: `número de épocas`,  `número de pasos de optimización por época`, `criterios de parada`,  lo que escribirá en el flufo de datos para seguimiento y evaluación (`writer`), y otras cosas.

## <span style="color:blue">Instalar Pytorch-lightning</span> 

En consola ejecute el siguiente comando. 

In [ ]:
#conda install -c conda-forge pytorch-lightning

## <span style="color:blue">Ejemplo de un módulo Lightning</span> 

Como ejemplo vamos a implementar nuevamente el auto-encoder para los datos de MNIST, usando Lightning.

### <span style="color:#4CC9F0">Importa módulos</span>

In [3]:
import torch
import pytorch_lightning as pl
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split

from torchvision.datasets.mnist import MNIST
from torchvision import transforms

### <span style="color:#4CC9F0">Clase Backbone: Nuestro modelo neuronal</span>

Definimos nuestra red neuronal con la clase *Backbone* que deriva de  la clase `torch.nn.Module`. Al hacer esto, se gana toda la implementacion disponible en la clase Definimos nuestra red neuronal con la clase *Backbone* que deriva de  la clase *torch.nn.Module*.

In [3]:
# modelo

class Backbone(torch.nn.Module):
    def __init__(self, hidden_dim=128):
        super().__init__()
        self.l1 = torch.nn.Linear(28 * 28, hidden_dim)
        self.l2 = torch.nn.Linear(hidden_dim, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.l1(x))
        x = torch.relu(self.l2(x))
        return x

### <span style="color:#4CC9F0">Clase  LitClassifier Nuestra clase Lightning</span>

Esta clase heredada de `pl.LightningModule`  nos permite implementar los métodos que usará el entrenador para entrenr nuestra red. Es necesario especificar:

* el método de cálculo d ela red (`forward`). En realidad podemos definir la red dentro de esta clase, pero para desacoplar la rede y hacerlo esta clase más genérica, mejor definimos el modleo por fuera. de esta clase pytorch-lightning;
* el paso de entrenamiento, en el cual adicionalmente escribimos en el log para Tensorboard;
* el paso de validación, que tambien escribe el en log para Tensorboard;
* el paso de prueba, que tambien escribe el en log para Tensorboard; 
* Se configura el optimizador

In [3]:
class LitClassifier(pl.LightningModule):
    def __init__(self, backbone, learning_rate=1e-3):
        super().__init__()
        
        # modelo
        self.backbone = backbone
        # métricas
        self.accuracy = torchmetrics.Accuracy()

    def forward(self, x):
        # use forward para inferencia/predicciones
        embedding = self.backbone(x)
        return embedding

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.backbone(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('train_loss', loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.backbone(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('valid_loss', loss, on_step=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.backbone(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('test_loss', loss)

    def training_epoch_end(self, outs):
        # log epoch metric
        self.log('train_acc_epoch', self.accuracy)
    
    def configure_optimizers(self):
        # self.hparams está disponible porque nosotros llamamos self.save_hyperparameters()
        return torch.optim.Adam(self.parameters(), lr=1.0e-3)

 

### <span style="color:#4CC9F0">Entrenador</span>

Instancia un entrenador

In [ ]:
#trainer = pl.Trainer(gpus=4, precisión=16, limit_train_batches=0.5)
trainer = pl.Trainer(accelerator='cpu', max_epochs= 10)

### <span style="color:#4CC9F0">Datos</span>

In [3]:
dataset = MNIST('', train=True, download=True, transform=transforms.ToTensor())
mnist_test = MNIST('', train=False, download=True, transform=transforms.ToTensor())
mnist_train, mnist_val = random_split(dataset, [55000, 5000])

train_loader = DataLoader(mnist_train, batch_size=args.batch_size)
val_loader = DataLoader(mnist_val, batch_size=args.batch_size)
test_loader = DataLoader(mnist_test, batch_size=args.batch_size)

### <span style="color:#4CC9F0">Entrenamiento</span>

In [3]:
hidden_dim = 128
model = LitClassifier(Backbone(hidden_dim=hidden_dim))
trainer.fit(model, train_loader, val_loader)

### <span style="color:#4CC9F0">Prueba</span>

In [3]:
result = trainer.test(test_dataloaders=test_loader)
print(result)